In [1]:
import requests
import os

from datetime import datetime, timedelta
import shutil

In [9]:
gfs_domain = 'https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/v16.3/'
day = '20230914'
ftimes = ['00'] #['00', '06', '12', '18']

storage_path = './data'

In [3]:
def get_url(domain, date, ftime):
    return f'{domain}gfs.{date}/{ftime}/wave/gridded/'

In [4]:
def fetch_file(file):
    storage_path, url = file
    r = requests.get(url, stream=True)
    with open(storage_path, 'wb') as f:
        for ch in r:
            f.write(ch)
    return True

In [5]:
def get_filenames(ftime):
    filenames = []

    for i in range(385):
        if i > 120 and i % 3 != 0:
            continue

        file_num = '{:03d}'.format(i)
        filename = f'gfswave.t{ftime}z.global.0p25.f{file_num}.grib2'

        filenames.append(filename)
        filenames.append(filename + '.idx')
    
    return filenames

In [6]:
for time in ftimes:
    os.makedirs(f'{storage_path}/{day}/{time}', exist_ok=True)

In [10]:
all_files = []

for ftime in ftimes:
    url = get_url(gfs_domain, day, ftime)
    filenames = get_filenames(ftime)
    files = [(f'{storage_path}/{day}/{ftime}/{filename}', url + filename) for filename in filenames]
    all_files += files

print('downloading...')
file_count = 0

for file in all_files:
    file_count += int(fetch_file(file))

print(f'Downloaded {file_count} files')

downloading...
Downloaded 418 files


In [8]:
def housekeeping(backlog_days):
    print(f'deleting folders for days older than {backlog_days} days..')
    last_date = (datetime.today() - timedelta(backlog_days)).strftime('%Y%m%d')
    try:
        folders = sorted(os.listdir(storage_path))
    except FileNotFoundError:
        print('folder is empty')
        return

    if last_date not in folders:
        folders = sorted(folders + [last_date])
    index = folders.index(last_date)

    for folder in folders[:index]:
        print(folder)
        shutil.rmtree(f'{storage_path}/{folder}')


housekeeping(7)

deleting folders for days older than 7 days..
